Install tensorflow gpu

In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 52kB/s 


In [2]:
!nvidia-smi

Wed Oct 28 08:10:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
tf.__version__

'2.3.1'

Kết nối tới google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Trước khi thực hiện train cần chạy một số modules này trước phục vục cho việc code

In [5]:
import os
import cv2
import sys
import zipfile

def fun_print(name: str, value) -> None:
    print('@ Deep Learning> ', name)
    print(value)


def fun_getFileNames(path: str) -> list:
    return os.listdir(path)


def fun_showVideoPath(path: str, delay: int = 25) -> None:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    while True:
        if not isContinue:
            break
        cv2.imshow('frame', frame)
        if cv2.waitKey(delay=delay) & 0xFF == ord('q'):
            break
        isContinue, frame = cap.read()

    cap.release()
    cv2.destroyAllWindows()


def fun_getFramesOfVideo(path: str, count: int = 20) -> list:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    imgs = []
    while count > 0:
        if not isContinue:
            break
        imgs.append(frame)
        isContinue, frame = cap.read()
        count -= 1
    cap.release()
    cv2.destroyAllWindows()
    return imgs

def fun_getFramesOfVideo_ALL(path: str) -> list:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    imgs = []
    while True:
        if not isContinue:
            break
        imgs.append(frame)
        isContinue, frame = cap.read()
    cap.release()
    cv2.destroyAllWindows()
    return imgs


def fun_showVideoFrames(frames: list, delay: int = 25) -> None:
    for frame in frames:
        cv2.imshow('frame', frame)
        if cv2.waitKey(delay=delay) & 0xFF == ord('q'):
            break


def fun_showVideo(source, delay: int = 25) -> None:
    if isinstance(source, str):
        fun_showVideoPath(path=source, delay=delay)
    else:
        fun_showVideoFrames(frames=source, delay=delay)


def fun_resizeFrames(frames: list, size: tuple = (224, 224)) -> list:
    imgs = []
    count = 0
    for frame in frames:
        try:
          fr = cv2.resize(frame, dsize=size)
        except:
          print('\r!Error To Resize of {0}'.format(count))
        imgs.append(fr)
        count += 1
    cv2.destroyAllWindows()
    return imgs


def fun_saveFramesToVideo(frames: list, path: str, fps: int = 25) -> bool:
    try:
        height, width, layer = frames[0].shape
        wr = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
        for frame in frames:
            wr.write(frame)
        wr.release()
        cv2.destroyAllWindows()
        return True

    except:
        fun_print(name='Write Video: '+path, value='ERROR TO WRITE VIDEO')
        return False

def fun_getSizeOfFrame(frame) -> tuple:
    height, width, layer = frame.shape
    return (width, height)

# version 1
def fun_outListVideoWithNumFrame(pathVideoLoad: str, dirToSave: str, preFixName: str, videoNameIndex: int= None, countFrame: int = 40, fps: int = 25, isShowCalculating: bool = False) -> int:
    if videoNameIndex is None:
        fun_print('fun_outListVideoWithNumFrame', 'Please input para: videoNameIndex')
        return 0

    all = 0
    countWriten = 0
    if isShowCalculating:
        fun_print('Calculator Video Out Frame', 'calculating...')
        all = fun_getFramesOfVideo_ALL(pathVideoLoad)
        all = len(all) // countFrame

    cap = cv2.VideoCapture(pathVideoLoad)
    isContinue, frame = cap.read()
    count = videoNameIndex
    while True:
        if not isContinue:
            break
        nameFile = dirToSave + preFixName + '_out_'+str(count)+'.avi'
        cFrame = countFrame
        frames = []

        # get list frame
        while cFrame > 0:
            frames.append(frame)
            isContinue, frame = cap.read()
            if not isContinue:
                break
            cFrame -= 1

        # check video enough frameCount frame ?
        if len(frames) != countFrame:
            break

        # write list frame
        res = fun_saveFramesToVideo(frames=frames, path=nameFile, fps=fps)
        countWriten += 1
        if res:
            if isShowCalculating:
                percent = countWriten / all
                mess = '\r - Writen: {0} -> Complete: {1:.1%}'.format(nameFile, percent)
                sys.stdout.write(mess)
                sys.stdout.flush()
            else:
                mess = '\r - Writen: {0} -> Complete'.format(nameFile)
                sys.stdout.write(mess)
                sys.stdout.flush()

        # done
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return count

def fun_extractZipFile(pathFileZip: str, pathToSave: str) -> None:
    if not os.path.exists(pathToSave):
        os.makedirs(pathToSave)
    fun_print(name='Extract All ' + pathFileZip, value='Extracting...')
    if (pathFileZip.endswith('.zip')):
        zipfile.ZipFile(file=pathFileZip, mode='r').extractall(path=pathToSave)
        print('Extract Done')
    else:
        print('Please Input zip file')

def fun_print_process(count: int, max: int, mess: str = 'Processing: ') -> None:
  process = count / max
  mess = '\r - ' +  mess + ' [{0:.1%}]'.format(process)
  sys.stdout.write(mess)
  sys.stdout.flush()

Mình cần định nghĩa một số Const để dể dàng thực hiện khi mỗi lần Retrain. TẠI ĐÂY

In [44]:
DIR_ROOT = '/content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc'
DIR_INPUT_TRAIN = DIR_ROOT + '/InputTrain_28_10_2020'
DIR_MODEL_LSTM = DIR_ROOT + '/Models/LSTM_Train_28_10_2020.h5'
DIR_MODEL_CNN = DIR_ROOT + '/Models/VGG16_Model.h5'
SIZE = (224, 224)
NUM_FRAME_INPUT_LSTM = 20
TRANSFER_VALUE_SIZE = 4096
RNN_SIZE = 512
EPOCH = 100
BATCH_SIZE = 150

VIDEO_NAMES = [
  'bc',
  'cq',
  'da',
  'dn',
  'kc',
  'lg',
  'lk',
  'na',
  'nc',
  'ne',
  'nt',
  'om',
  'tc',
  'vd',
  'vk',
  'xd',
  'xt'
]

VIDEO_LABELS = [
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
]

NUM_CLASSIFY = len(VIDEO_NAMES)

Mình cần phải có dữ liệu để train, dữ liệu thường ở dưới máy local, phải tải lên google drive để dùng được cho colab,
dữ liệu gồm nhiều files khi tải lên google drive sẽ bất tiện. Mình cần nén nó lại thành file [*.zip] rồi tải lên 
google drive. Xong sau đó dùng code này để giải nén zip ra thì mình sẽ có dữ liệu,
Sau khi mình tải tệp zip lên google drive thì bắt đầu công việc giải nén ra một thư để thực hiện việt input train

In [9]:
fun_extractZipFile(pathFileZip= DIR_ROOT + '/InputsZip/Train_28_10_2020.zip', pathToSave= DIR_INPUT_TRAIN)

@ Deep Learning>  Extract All /content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc/InputsZip/Train_28_10_2020.zip
Extracting...
Extract Done


Bây giờ để đảm bảo rằng tất cả các file của chúng ta được giải nén thành công vào trong thư mục mình mong muốn. Mình cần thực hiện dòng lệnh này để xem.

In [18]:
fileNames = fun_getFileNames(path= DIR_INPUT_TRAIN)
fun_print(name= 'Xem thư mục bao nhiêu file?: ' + DIR_INPUT_TRAIN, value= 'Có: {0} Files'.format(len(fileNames)))

@ Deep Learning>  Xem thư mục bao nhiêu file?: /content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc/InputTrain_28_10_2020
Có: 17 Files


Dữ liệu của mình đã có. Bây giờ bắt đầu giai đoạn tiếp theo. Load model VGG16

Đầu là khu vực dành cho import thư viện

In [11]:
from keras.applications import VGG16
from keras.models import Model, Sequential, load_model
import numpy as np
from random import shuffle
from keras.layers import LSTM, Dense, Activation

In [14]:
#modelCNN = VGG16(include_top= True, weights= 'imagenet')
modelCNN = load_model(DIR_MODEL_CNN, compile= False)
modelCNN.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Mình sẽ thử dùng modelCNN để predict 20 frame ảnh xem sao.

In [19]:
frames = fun_getFramesOfVideo(path= DIR_INPUT_TRAIN + '/bc/bc1_111_001.avi', count=20)

fun_print('count frame: ', value=len(frames))

transferLayer = modelCNN.get_layer(name='fc2')

imgModelTransfer = Model(inputs= modelCNN.input, outputs= transferLayer.output)

frames = fun_resizeFrames(frames= frames, size=SIZE)

frames = np.array(frames)
frames = (frames / 255.).astype(np.float16)

transfer = imgModelTransfer.predict(frames)
print(transfer)

@ Deep Learning>  count frame: 
20
[[0.         0.         1.5306206  ... 0.         0.         0.74161446]
 [0.         0.         1.5450199  ... 0.         0.         0.8388475 ]
 [0.         0.         1.6016072  ... 0.         0.         0.8584479 ]
 ...
 [0.         0.         1.6011251  ... 0.         0.         0.85297966]
 [0.         0.         1.6245921  ... 0.         0.         0.87145877]
 [0.         0.         1.7206725  ... 0.         0.         0.9652164 ]]


Bây giờ thực thực hiện định nghĩa một hàm. Nhận vào danh sách các video train. Trả về mảng tên video và mảng labels. Phần code này là dùng cho tất cả video nằm trong 1 thư mục

In [ ]:
def getIndexOF_VIDEO_NAMES(fileName: str) -> int:
  index = 0
  for txt in VIDEO_NAMES:
    if fileName.find(txt) != -1:
      return index
    index += 1
  return -1

def getVideoLabelNames(path: str):
  names = []
  labels = []
  fileNames = fun_getFileNames(path= path)
  vName = ''
  for file in fileNames:
    lableIndex = getIndexOF_VIDEO_NAMES(fileName= file)
    if lableIndex == -1:
      continue
    names.append(file)
    labels.append(VIDEO_LABELS[lableIndex])
  
  c = list(zip(names, labels))
  shuffle(c)

  names, labels = zip(*c)
  return names, labels

Phần code này cũng là để load video và trả về tập names và labels. Nhưng dùng cho phần video nằm trong từng thư mục.

In [51]:
def getVideoLabelNames_EachFolder(path: str):
  names = []
  labels = []
  
  for fol in VIDEO_NAMES:
    folder = path + '/' + fol
    fileNames = fun_getFileNames(path= folder)
    index = VIDEO_NAMES.index(fol)
    for file in fileNames:
      names.append('/' + fol + '/' + file)
      labels.append(VIDEO_LABELS[index])
  
  c = list(zip(names, labels))
  shuffle(c)

  names, labels = zip(*c)
  return names, labels

Bây giờ test thử hàm getVideoLabelNames(path: str): xem sao

In [52]:
names, labels = getVideoLabelNames_EachFolder(path= DIR_INPUT_TRAIN)

fun_print('Names', names[0:3])
fun_print('labels', labels[0:3])

@ Deep Learning>  Names
('/dn/dn8_030_002+10F.avi', '/om/om1_123_002.avi', '/vd/vd3_055_018.avi')
@ Deep Learning>  labels
([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])


Bây giờ mình sẽ viết một hàm nhận vào một video. Hàm này sẽ lấy 20 frame hình cho vào VGG16 để lấy mẫu dự đoán

In [26]:
def getTransferValue(pathVideoOrListFrame):
  if isinstance(pathVideoOrListFrame, str):
    frames = fun_getFramesOfVideo(path=pathVideoOrListFrame, count=NUM_FRAME_INPUT_LSTM)
  else:
    frames = pathVideoOrListFrame
    
  frames = fun_resizeFrames(frames= frames, size=SIZE)

  frames = np.array(frames)
  frames = (frames / 255.).astype(np.float16)

  transfer = imgModelTransfer.predict(frames)
  return transfer

Cùng test thử getTransferValue(pathVideo: str): xem nào!

In [28]:
transfer = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + '/bc/bc1_111_001.avi')
print(transfer)

[[0.         0.         1.5306206  ... 0.         0.         0.74161446]
 [0.         0.         1.5450199  ... 0.         0.         0.8388475 ]
 [0.         0.         1.6016072  ... 0.         0.         0.8584479 ]
 ...
 [0.         0.         1.6011251  ... 0.         0.         0.85297966]
 [0.         0.         1.6245921  ... 0.         0.         0.87145877]
 [0.         0.         1.7206725  ... 0.         0.         0.9652164 ]]


Bây giờ mình sẽ định nghĩa một hàm nhận vào một nhãn của video (label) trả về một dạng ones hot

In [29]:
def onesHotLabel(label: list):
  _ones = np.ones([NUM_FRAME_INPUT_LSTM, NUM_CLASSIFY])
  _onesHot = label * _ones
  return np.array(_onesHot)

Test thử hàm oneHotLabel(label: list): xem nào!



In [31]:
_oneHot = onesHotLabel([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
fun_print(name='oneHost', value= _oneHot)

@ Deep Learning>  oneHost
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [

Từ nãy giờ cũng được nhiều rồi. Để đảm bảo rằng dữ liệu hợp lệ. Bạn hãy test lại một lần nửa xem đã có dữ liệu chưa. Hãy thực hiện các lệnh dưới đây.

In [34]:
names, labels = getVideoLabelNames_EachFolder(path= DIR_INPUT_TRAIN)

fun_print(name= 'Size of List video', value= len(names))
fun_print(name= 'Size of List labels', value= len(labels))

print('= '*50)

print(names[0:5])
print(labels[0:5])

print('= '*50)

print(getTransferValue(DIR_INPUT_TRAIN + names[0]))
print(onesHotLabel(labels[0]))

@ Deep Learning>  Size of List video
1255
@ Deep Learning>  Size of List labels
1255
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
('/da/da5_107_001.avi', '/dn/dn10_143_007.avi', '/om/om10_134_005.avi', '/vd/vd3_055_017.avi', '/vk/vk1_125_002.avi')
([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
[[0.         0.         1.294548   ... 0.         0.         0.7627448 ]
 [0.         0.         1.2842535  ... 0.         0.         0.7733822 ]
 [0.         0.         1.3218735  ... 0.         0.         0.76401573]
 ...
 [0.         0.         1.3335023  ... 0.         0.         0.85251784]
 [0.         0.         1.300679

Nếu mọi thứ đã Ok bây giờ mình sẽ viết một hàm nhận vào số lượng video muốn train. Trả về 2 danh sách:
- TranSet: tập dữ liệu train
- LabelSet: tập dữ liệu gán nhãn cho mỗi tập trong trainSet.

Nếu mọi thứ thuận lợi thì nó là tiền đề để bắt đầu huấn luyện LSTM.

In [35]:
def getTrainSet_LabelSet(numItem: int):
  count = 0
  trainSet = []
  labelSet = []
  while count < numItem:
    itemTrain = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + names[count])
    itemLable = onesHotLabel(label= labels[count])

    trainSet.append(itemTrain)
    labelSet.append(itemLable[0])

    fun_print_process(count= count, max= numItem)

    count += 1
  
  return trainSet, labelSet

Cùng Test Thử getTrainSet_LabelSet(numItem: int): xem nào!

In [36]:
trainSet, labelSet = getTrainSet_LabelSet(numItem= len(names))

 - Processing:  [99.9%]

In [37]:
print(len(trainSet))
print(len(trainSet[0]))
print(len(trainSet[0][0]))

1255
20
4096


In [38]:
print(len(labelSet))
print(labelSet[0])
print(labelSet[0][0])

1255
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0


Hãy viết một hàm trả về cấu trúc model LSTM

In [41]:
def getModelLSTM(rnn_size: int = 512, input_shape: tuple = (20, 4096), num_classify: int = 3):
  modelLSTM = Sequential()
  modelLSTM.add(LSTM(rnn_size, input_shape= input_shape))
  modelLSTM.add(Dense(1024))
  modelLSTM.add(Activation('relu'))
  modelLSTM.add(Dense(50))
  modelLSTM.add(Activation('sigmoid'))
  modelLSTM.add(Dense(num_classify))
  modelLSTM.add(Activation('softmax'))
  modelLSTM.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
  return modelLSTM

In [42]:
model = getModelLSTM(num_classify= NUM_CLASSIFY)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               9439232   
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                51250     
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                867       
_________________________________________________________________
activation_2 (Activation)    (None, 17)                0

In [45]:
model.fit(np.array(trainSet), np.array(labelSet), epochs=EPOCH, batch_size=BATCH_SIZE, verbose=2, validation_split= 0.2)

model.save(DIR_ROOT + '/Models/LSTM_Train_28_10_2020.h5')

fun_print(name= 'LSTM Train', value= 'Finish')

Epoch 1/100
7/7 - 1s - loss: 0.0335 - accuracy: 0.5767 - val_loss: 0.0489 - val_accuracy: 0.3466
Epoch 2/100
7/7 - 1s - loss: 0.0339 - accuracy: 0.5827 - val_loss: 0.0460 - val_accuracy: 0.4143
Epoch 3/100
7/7 - 1s - loss: 0.0341 - accuracy: 0.5767 - val_loss: 0.0462 - val_accuracy: 0.4024
Epoch 4/100
7/7 - 1s - loss: 0.0336 - accuracy: 0.5797 - val_loss: 0.0457 - val_accuracy: 0.4303
Epoch 5/100
7/7 - 1s - loss: 0.0329 - accuracy: 0.5926 - val_loss: 0.0472 - val_accuracy: 0.3865
Epoch 6/100
7/7 - 1s - loss: 0.0328 - accuracy: 0.5906 - val_loss: 0.0457 - val_accuracy: 0.4382
Epoch 7/100
7/7 - 1s - loss: 0.0332 - accuracy: 0.5896 - val_loss: 0.0468 - val_accuracy: 0.4024
Epoch 8/100
7/7 - 1s - loss: 0.0336 - accuracy: 0.5827 - val_loss: 0.0475 - val_accuracy: 0.3865
Epoch 9/100
7/7 - 1s - loss: 0.0342 - accuracy: 0.5847 - val_loss: 0.0476 - val_accuracy: 0.3944
Epoch 10/100
7/7 - 1s - loss: 0.0334 - accuracy: 0.5956 - val_loss: 0.0456 - val_accuracy: 0.4143
Epoch 11/100
7/7 - 1s - loss:

Bây giờ hãy cùng test xem độ chính xác của model nào.

In [53]:
def getNameLabelsVal(count: int):
  return names[0: count], labels[0: count]

In [55]:
namesVald, labelsVald = getNameLabelsVal(100)

print(namesVald)
print(len(namesVald))

('/dn/dn8_030_002+10F.avi', '/om/om1_123_002.avi', '/vd/vd3_055_018.avi', '/da/da7_186_001.avi', '/nt/nt5_108_001_20F.avi', '/nc/nc10_134_001.avi', '/dn/dn10_134_005.avi', '/ne/ne7_194_006.avi', '/lk/lk8_024_003.avi', '/dn/dn8_026_002.avi', '/dn/dn2_012_001.avi', '/xd/xd8_023_002.avi', '/dn/dn7_192_003.avi', '/ne/ne3_047_001.avi', '/da/da8_035_002.avi', '/dn/dn6_205_008.avi', '/kc/kc9_167_001.avi', '/lg/lg2_009_003.avi', '/dn/dn6_201_001.avi', '/dn/dn9_166_002_15F.avi', '/xt/xt6_199_001_30f.avi', '/xt/xt3_050_008.avi', '/lk/lk7_177_001_14F.avi', '/om/om5_104_002.avi', '/vd/vd3_048_002.avi', '/vk/vk3_045_004.avi', '/vk/vk3_046_001.avi', '/om/om5_091_003.avi', '/dn/dn10_138_019.avi', '/da/da7_197_001.avi', '/lg/lg7_196_001.avi', '/dn/dn1_123_005.avi', '/dn/dn6_205_004.avi', '/xd/xd3_056_001.avi', '/xd/xd7_191_002.avi', '/da/da8_034_0071.avi', '/kc/kc8_029_004.avi', '/ne/ne3_052_011.avi', '/lg/lg8_034_001.avi', '/vk/vk8_026_001.avi', '/nt/nt10_138_003.avi', '/nt/nt4_072_002.avi', '/om/om8

In [59]:
def getValdSet_ValdLabelSet(numItem: int):
  count = 0
  valdSet = []
  valdLabel = []
  while count < numItem:
    itemVald = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + namesVald[count])
    itemLabel = onesHotLabel(label= labelsVald[count])

    valdSet.append(itemVald)
    valdLabel.append(itemLabel[0])

    fun_print_process(count= count, max= numItem)
    count += 1

  return valdSet, valdLabel

In [60]:
valdSet, valdLabelSet = getValdSet_ValdLabelSet(numItem= len(namesVald))

 - Processing:  [99.0%]

In [61]:
result = model.evaluate(np.array(valdSet), np.array(valdLabelSet))

4/4 [==============================] - 0s 14ms/step - loss: 0.0352 - accuracy: 0.5500


In [62]:
for name, value in zip(model.metrics_names, result):
  print(name, value)

loss 0.03522932156920433
accuracy 0.550000011920929


Hãy cùng thử predict với từng video và xem kết quả nào

In [63]:
for file in namesVald:
  frames = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + file)
  pre = model.predict(np.array([frames]))
  fun_print(name= 'predict: ' + file + ' | {0}'.format(np.argmax(pre)), value= pre)
  print('\r')

@ Deep Learning>  predict: /dn/dn8_030_002+10F.avi | 3
[[1.18904293e-03 7.21564458e-04 2.89783645e-02 7.54066944e-01
  8.01245682e-03 4.25280184e-02 8.55987612e-03 1.15650995e-02
  8.02562293e-03 6.07429363e-04 2.16689184e-02 2.22636722e-02
  2.71137280e-04 1.44017104e-03 2.83728656e-03 2.27394048e-02
  6.45249337e-02]]

@ Deep Learning>  predict: /om/om1_123_002.avi | 11
[[0.00564696 0.00099151 0.00165794 0.26298362 0.01539241 0.005252
  0.02653017 0.10766509 0.00634273 0.0113151  0.04744524 0.2832051
  0.01585857 0.00340945 0.0096161  0.18441096 0.01227707]]

@ Deep Learning>  predict: /vd/vd3_055_018.avi | 13
[[4.2883641e-04 1.0985852e-04 1.5954213e-03 2.3801650e-03 3.3538940e-03
  1.2494946e-03 3.4331023e-03 6.9849081e-02 1.2842685e-03 8.5208472e-03
  1.5034587e-02 8.6867595e-03 2.9956771e-04 8.6696815e-01 1.2559790e-02
  4.2231400e-03 2.3004462e-05]]

@ Deep Learning>  predict: /da/da7_186_001.avi | 2
[[1.7810630e-03 5.8537110e-04 3.5226464e-01 1.0526584e-01 1.9591875e-02
  2.4286

Bây giờ hãy thử put text và save video về thử xem.

In [64]:
def fun_predict(model, tranfer_value):
    arrPre = []
    arrPre.append(tranfer_value)
    Real = model.predict(np.array(arrPre))
    pre = np.argmax(Real)

    print(Real, pre)
    print('\r')
    return pre, Real[0][pre]

#version 1
def fun_loadVideoPredictPutTextAndSave(modelCNN, modelLSTM, pathLoadVideo, pathSave, perFameInputLSTM= 20, tranferSize: int = 4096):
    if pathLoadVideo is None or pathSave is None or perFameInputLSTM is None or modelCNN is None:
        return

    cap = cv2.VideoCapture(pathLoadVideo)
    isContinue, frame = cap.read()

    size = fun_getSizeOfFrame(frame= frame)
    wr = cv2.VideoWriter(pathSave, cv2.VideoWriter_fourcc(*'MJPG'), 25, size)
    countFrame = 0

    while True:
      if not isContinue:
        break
      
      imgs = []
      trans = []
      count = perFameInputLSTM

      while count > 0:
        imgs.append(frame)
        trans.append(frame)
        isContinue, frame = cap.read()
        count -= 1

      if len(imgs) != perFameInputLSTM:
        break

      trans = getTransferValue(trans)

      pre, real = fun_predict(model= model, tranfer_value= trans)

      color=(100, 200, 255)
      conv = ''
      if pre == 0:
        conv = 'Danh'
        color = (150, 250, 255)
      elif pre == 1:
        color = (200, 100, 255)
        conv = 'Da'
      elif pre == 2:
        conv = 'Nam Toc'

      text = 'Predict: {0} -> Real: [ {1} ]'.format(conv, real)

      for ff in imgs:
        countFrame += 1
        # putText
        cv2.putText(img=ff,
                    text=text,
                    org=(50, 100),
                    fontScale=1,
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    thickness=2,
                    color= color )

        cv2.putText(img=ff,
                    text='Frame Count: {0}'.format(countFrame),
                    org=(50, 150),
                    fontScale=1,
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    thickness=2,
                    color= color )

        wr.write(ff)

      isContinue, frame = cap.read()

    wr.release()
    cap.release()
    cv2.destroyAllWindows()


In [65]:
model = load_model(DIR_MODEL_LSTM)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               9439232   
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                51250     
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                867       
_________________________________________________________________
activation_2 (Activation)    (None, 17)                0

In [67]:
fun_loadVideoPredictPutTextAndSave(modelCNN= imgModelTransfer, modelLSTM= model, pathLoadVideo= DIR_ROOT + '/Tests/video_test7.mp4',
                                   pathSave= DIR_ROOT + '/Tests/video_test7_out.avi', perFameInputLSTM= 20)

[[5.1347916e-03 2.8760213e-04 8.9627542e-03 5.2117974e-01 2.7546152e-02
  1.4756451e-02 2.1180442e-02 1.9527458e-02 1.0487816e-02 2.2281215e-03
  1.3201849e-01 3.5049573e-02 2.0131436e-03 1.9452341e-02 9.0801023e-02
  8.8718362e-02 6.5577065e-04]] 3

[[0.00623023 0.00037562 0.0197699  0.30963737 0.03679128 0.02987615
  0.02458843 0.01926603 0.01150506 0.00189071 0.20597608 0.03447659
  0.00243176 0.02998772 0.1816248  0.08525355 0.00031867]] 3

[[5.3246114e-03 5.0269748e-04 7.6957978e-04 7.9410893e-01 3.9006770e-03
  2.7323137e-03 8.4082959e-03 1.8746592e-02 4.3715099e-03 2.3766230e-03
  1.7983833e-02 4.8743468e-02 6.1608087e-03 4.2364647e-04 4.3700156e-03
  6.8270199e-02 1.2806201e-02]] 3

[[3.26471403e-03 2.42431182e-04 3.65992222e-04 9.17422414e-01
  1.61015405e-03 1.39439001e-03 2.86772684e-03 5.13475249e-03
  2.59467610e-03 7.36758811e-04 6.16489444e-03 1.68433506e-02
  2.76269787e-03 1.12953734e-04 1.95947126e-03 3.02971862e-02
  6.22533727e-03]] 3

[[0.01090094 0.00070763 0.0026

**End**

---

